# Вводный курс по Big Data (семинары)

## Урок 2. SQL & Big Data

Датасет по ценам на жилье Airbnb, доступный на kaggle.com: (https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data)
            
Расчет среднего значения и дисперсии по признаку ”price” методом mapreduce

In [1]:
import pandas as pd

estates = pd.read_csv("archive/AB_NYC_2019.csv", engine='python', encoding='utf-8', on_bad_lines="skip", encoding_errors="ignore")
estates.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [2]:
prices_series = estates['price']
prices_series.info()
prices = prices_series.tolist()

<class 'pandas.core.series.Series'>
RangeIndex: 48895 entries, 0 to 48894
Series name: price
Non-Null Count  Dtype
--------------  -----
48895 non-null  int64
dtypes: int64(1)
memory usage: 382.1 KB


In [3]:
from functools import reduce

def mapper(price):
    return (price, )

def reducer(price_data1, price_data2):
        prices = []
        if len(price_data1) == 1:
            n, mean, delta_sqr = 0, 0.0, 0
            prices.append(price_data1[0])
        else:
            n, mean, delta_sqr = price_data1
        prices.append(price_data2[0])
        for price in prices:
            n += 1
            delta = price - mean
            mean += delta / n
            delta_sqr += delta * (price - mean)
        return n, mean, delta_sqr

In [4]:
from joblib import Parallel, delayed
import math as m

n, mean, delta_sqr = reduce(reducer, map(mapper, prices))
variance = delta_sqr / n
sdv = variance ** (1/2)
print(f'Среднее: {mean}, дисперсия: {variance}, СКО: {sdv}')


Среднее: 152.72068718682823, дисперсия: 57672.845698433375, СКО: 240.15171391941672


In [5]:
# Для проверки:
def mean(prices):
    return sum(prices)/len(prices)


def variance(prices):
    m = mean(prices)
    return sum([(price - m)**2 for price in prices])/len(prices)

print(f'Среднее: {mean(prices)}, дисперсия: {variance(prices)}')

Среднее: 152.7206871868289, дисперсия: 57672.84569843345


In [6]:
prices_series.to_csv('prices.csv', header=False)